In [27]:
import pandas as pd

# Load the merged TSV file
df = pd.read_csv("taxonomic_profiles_3.tsv", sep="\t")

# Rename columns by removing '_profile' suffix (skip the first column)
df.columns = [col.replace('_profile', '') for col in df.columns]

# Display the first 5 rows and first 10 columns
df.iloc[:5, :10]

,Feature\Sample,CSM5FZ3N_P,CSM5FZ3R_P,CSM5FZ3T_P,CSM5FZ3V_P,CSM5FZ3X_P,CSM5FZ3Z_P,CSM5FZ42_P,CSM5FZ44_P,CSM5FZ46_P
0,UNKNOWN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,k__Archaea,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,k__Archaea|p__Euryarchaeota,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,k__Archaea|p__Euryarchaeota|c__Methanobacteria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,k__Archaea|p__Euryarchaeota|c__Methanobacteria...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
def extract_taxa_level(df, tax_level="genus"):
    """
    Extract genus- or species-level rows from a MetaPhlAn merged table,
    with unique taxonomy labels (prepend parent if duplicated).

    Parameters:
    - df (pd.DataFrame): Loaded MetaPhlAn dataframe.
    - tax_level (str): "genus" or "species".

    Returns:
    - pd.DataFrame: Subsetted and renamed dataframe.
    """
    # Define regex filters
    if tax_level == "genus":
        tax_filter = df.iloc[:, 0].str.contains(r'\|g__', na=False) & ~df.iloc[:, 0].str.contains(r'\|s__', na=False)
    elif tax_level == "species":
        tax_filter = df.iloc[:, 0].str.contains(r'\|s__', na=False)
    else:
        raise ValueError("tax_level must be 'genus' or 'species'")

    df_tax = df[tax_filter].copy()

    # Extract taxonomy label (g__ or s__) and optional parent level
    df_tax['tax_name'] = df_tax.iloc[:, 0].str.extract(rf'{tax_level[0]}__([^|]*)')[0]

    if tax_level == "genus":
        df_tax['parent'] = df_tax.iloc[:, 0].str.extract(r'f__([^|]*)')[0]
    elif tax_level == "species":
        df_tax['parent'] = df_tax.iloc[:, 0].str.extract(r'g__([^|]*)')[0]

    # Handle duplicates by prepending parent
    duplicated = df_tax['tax_name'].duplicated(keep=False)
    df_tax['taxonomy_label'] = df_tax.apply(
        lambda row: f"{row['parent']}:{row['tax_name']}" if duplicated[row.name] else row['tax_name'],
        axis=1
    )

    # Replace original taxonomy column
    df_tax.iloc[:, 0] = df_tax['taxonomy_label']
    df_tax.drop(columns=['tax_name', 'parent', 'taxonomy_label'], inplace=True)

    return df_tax



In [29]:
def filter_by_abundance(df, abundance_threshold=0.01, sample_fraction_threshold=0.10, output_path=None):
    """
    Filter taxa with abundance > threshold in > fraction of samples.

    Parameters:
    - df (pd.DataFrame): Dataframe with taxa as rows and samples as columns.
    - abundance_threshold (float): Minimum abundance (e.g., 0.01 for 0.01%).
    - sample_fraction_threshold (float): Minimum fraction of samples (e.g., 0.1 for 10%).
    - output_path (str, optional): If provided, saves the filtered table to this path.

    Returns:
    - pd.DataFrame: Filtered dataframe.
    """
    abundance_matrix = df.iloc[:, 1:].astype(float)
    n_samples = abundance_matrix.shape[1]
    pass_filter = (abundance_matrix > abundance_threshold).sum(axis=1) > (sample_fraction_threshold * n_samples)

    df_filtered = df[pass_filter]

    if output_path:
        df_filtered.to_csv(output_path, sep="\t", index=False)

    return df_filtered


In [30]:
df_genus = extract_taxa_level(df, tax_level="genus")

df_genus.shape[0]

187

In [31]:
df_genus.to_csv("taxonomic_profiles_3_genus.tsv", sep="\t", index=False)

In [32]:
df_genus_filter = filter_by_abundance(df_genus, abundance_threshold=0.01, sample_fraction_threshold=0.10, output_path=None)

In [33]:
df_genus_filter.shape[0]

58

In [34]:
df_genus_filter.to_csv("taxonomic_profiles_3_genus_filter.tsv", sep="\t", index=False)

In [35]:
df_species = extract_taxa_level(df, tax_level="species")
df_species.to_csv("taxonomic_profiles_3_species.tsv", sep="\t", index=False)

df_species.shape[0]

578

In [36]:
df_species_filter = filter_by_abundance(df_species, abundance_threshold=0.01, sample_fraction_threshold=0.10, output_path=None)
df_species_filter.to_csv("taxonomic_profiles_3_species_filter.tsv", sep="\t", index=False)
df_species_filter.shape[0]

107

# Pathabundances

In [15]:
import pandas as pd
# Load the merged TSV file
path_df = pd.read_csv("pathabundances_3.tsv", sep="\t")

# Rename columns by removing '_profile' suffix (skip the first column)
path_df.columns = [col.replace('_pathabundance_cpm', '') for col in path_df.columns]

# Display the first 5 rows and first 10 columns
path_df.iloc[:5, :10]

,Feature\Sample,CSM5FZ3N_P,CSM5FZ3R_P,CSM5FZ3T_P,CSM5FZ3V_P,CSM5FZ3X_P,CSM5FZ3Z_P,CSM5FZ42_P,CSM5FZ44_P,CSM5FZ46_P
0,UNMAPPED,334835.0,314137.0,211847.0,258470.0,262821.0,243972.0,248419.0,259163.0,252207.0
1,UNINTEGRATED,626270.0,645653.0,754069.0,697813.0,699100.0,713431.0,712841.0,703675.0,707898.0
2,UNINTEGRATED|g__Absiella.s__Absiella_dolichum,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,UNINTEGRATED|g__Acidaminococcus.s__Acidaminoco...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,UNINTEGRATED|g__Acidaminococcus.s__Acidaminoco...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
# Remove rows where the first column contains 'UNINTEGRATED'
path_df = path_df[~path_df.iloc[:, 0].str.contains("UNINTEGRATED", na=False)].reset_index(drop=True)
path_df = path_df[~path_df.iloc[:, 0].str.contains("UNMAPPED", na=False)].reset_index(drop=True)

# Display the first 5 rows and first 10 columns
path_df.iloc[:5, :10]

,Feature\Sample,CSM5FZ3N_P,CSM5FZ3R_P,CSM5FZ3T_P,CSM5FZ3V_P,CSM5FZ3X_P,CSM5FZ3Z_P,CSM5FZ42_P,CSM5FZ44_P,CSM5FZ46_P
0,1CMET2-PWY: N10-formyl-tetrahydrofolate biosyn...,430.803,467.145,452.468,476.92200,407.538,467.05,464.769,462.125,477.567
1,1CMET2-PWY: N10-formyl-tetrahydrofolate biosyn...,0.000,0.000,0.000,0.00000,0.000,0.00,0.000,0.000,0.000
2,1CMET2-PWY: N10-formyl-tetrahydrofolate biosyn...,0.000,0.000,0.000,0.00000,0.000,0.00,0.000,0.000,0.000
3,1CMET2-PWY: N10-formyl-tetrahydrofolate biosyn...,0.000,0.000,0.000,3.71367,0.000,0.00,0.000,0.000,0.000
4,1CMET2-PWY: N10-formyl-tetrahydrofolate biosyn...,0.000,0.000,0.000,0.00000,0.000,0.00,0.000,0.000,0.000


In [18]:
path_df.shape[0]

21681

In [19]:
# Assume the pathway names are in the first column
# Unstratified rows: no taxonomic info (no '|' character)
unstratified_df = path_df[~path_df.iloc[:, 0].str.contains(r'\|', na=False)].copy()

unstratified_df.shape[0]

476

In [42]:
# Split feature column and numeric data
features = unstratified_df.iloc[:, 0]
abundances = unstratified_df.iloc[:, 1:].astype(float)

# Convert to relative abundances (percent per sample)
rel_abundances = abundances.div(abundances.sum(axis=0), axis=1) * 100

# Combine with the feature labels
unstratified_df_rel_abd = pd.concat([features, rel_abundances], axis=1)


In [43]:
unstratified_df_filter = filter_by_abundance(unstratified_df_rel_abd, \
                                             abundance_threshold=0.01, \
                                             sample_fraction_threshold=0.10, output_path=None)
unstratified_df_filter.shape[0]

334

In [44]:
unstratified_df_filter.iloc[:5, :10]

,Feature\Sample,CSM5FZ3N_P,CSM5FZ3R_P,CSM5FZ3T_P,CSM5FZ3V_P,CSM5FZ3X_P,CSM5FZ3Z_P,CSM5FZ42_P,CSM5FZ44_P,CSM5FZ46_P
0,1CMET2-PWY: N10-formyl-tetrahydrofolate biosyn...,1.107608,1.161743,1.327509,1.090935,1.070247,1.096435,1.199713,1.243527,1.197085
213,ANAEROFRUCAT-PWY: homolactic fermentation,0.617621,0.516243,0.491389,0.521490,0.464173,0.548753,0.479205,0.474143,0.533998
241,ANAGLYCOLYSIS-PWY: glycolysis III (from glucose),0.951518,0.924192,1.045711,0.948376,0.838554,0.930039,0.973118,1.053321,0.988525
323,ARG+POLYAMINE-SYN: superpathway of arginine an...,0.000000,0.019752,0.050296,0.032625,0.141106,0.135872,0.147952,0.036195,0.081265
337,"ARGDEG-PWY: superpathway of L-arginine, putres...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [45]:
unstratified_df_filter.to_csv("pathabundances_3_unstratified_filter.tsv", sep="\t", index=False)

In [46]:

# Stratified rows: contain taxonomic info
stratified_df = path_df[path_df.iloc[:, 0].str.contains(r'\|', na=False)].copy()

stratified_df.shape[0]

21205

In [47]:
# Split feature column and numeric data
features = stratified_df.iloc[:, 0]
abundances = stratified_df.iloc[:, 1:].astype(float)

# Convert to relative abundances (percent per sample)
rel_abundances = abundances.div(abundances.sum(axis=0), axis=1) * 100

# Combine with the feature labels
stratified_df_rel_abd = pd.concat([features, rel_abundances], axis=1)


In [48]:
stratified_df_filter = filter_by_abundance(stratified_df_rel_abd, abundance_threshold=0.01, \
                                           sample_fraction_threshold=0.10, output_path=None)
stratified_df_filter.shape[0]

2724

2724 is still too large

In [49]:
stratified_df_filter = filter_by_abundance(stratified_df_rel_abd, abundance_threshold=0.1, \
                                           sample_fraction_threshold=0.10, output_path=None)
stratified_df_filter.shape[0]

653

In [50]:
stratified_df_filter.to_csv("pathabundances_3_stratified_filter.tsv", sep="\t", index=False)

# ECS

In [51]:
import pandas as pd
# Load the merged TSV file
ecs_df = pd.read_csv("ecs_3.tsv", sep="\t")

# Rename columns by removing '_profile' suffix (skip the first column)
ecs_df.columns = [col.replace('_pathabundance_cpm', '') for col in path_df.columns]

# Display the first 5 rows and first 10 columns
ecs_df.iloc[:5, :10]

/tmp/ipykernel_32264/486611580.py:3: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,

ValueError: Length mismatch: Expected axis has 1636 elements, new values have 1639 elements